In [56]:
import os 
from tensorflow import keras
import tensorflow as tf
import shutil
from matplotlib import pyplot as plt
from PIL import Image
import numpy as np

from keras.applications.inception_v3 import preprocess_input

In [17]:
PATH_TO_VANILLA_MODEL = '/home/odyssey/mmk_smoke_detection/model/hatches.h5'
PATH_TO_DATASET = './not_expanded_val'
LABELS = ['background', 'emission', 'fire', 'machine']

In [18]:
!ls /home/odyssey/mmk_smoke_detection/model/

hatches.h5  hatches.h5.dvc


In [19]:
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)
vanilla_model = keras.models.load_model(PATH_TO_VANILLA_MODEL, compile=True)

In [20]:
!nvidia-smi

Tue Oct 19 19:20:09 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.57.02    Driver Version: 470.57.02    CUDA Version: 11.4     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ...  On   | 00000000:01:00.0 Off |                  N/A |
| N/A   72C    P0    79W /  N/A |   4933MiB /  7973MiB |    100%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [21]:
N, M = 75, 75  # размер входного изображения


In [54]:
keras_ds = tf.keras.preprocessing.image_dataset_from_directory(
    directory=PATH_TO_DATASET,
    labels='inferred', 
    label_mode='int',
    class_names=LABELS, 
    batch_size=32, 
    image_size=(N, M)
)
keras_ds = keras_ds.prefetch(buffer_size=tf.data.experimental.AUTOTUNE)

Found 655 files belonging to 4 classes.


In [71]:
np.append(np.array([1,2]), np.array([2,4]))

array([1, 2, 2, 4])

In [76]:
preds = np.ndarray([], dtype=int)
y = np.ndarray([], dtype=int)

for batch in keras_ds:
    batch_X, batch_y = batch
    batch_X = preprocess_input(batch_X)
    batch_y = np.array(batch_y)
    y = np.append(y, batch_y)
    predictions = np.array(vanilla_model.predict_on_batch(batch_X))
    preds = np.append(preds, np.argmax(predictions, axis=1).astype(int))


In [81]:
from sklearn.metrics import classification_report
classification_report(y_true=y, y_pred=preds, output_dict=True)

/home/odyssey/.local/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/odyssey/.local/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/odyssey/.local/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


{'0': {'precision': 0.5138339920948617,
  'recall': 0.8074534161490683,
  'f1-score': 0.6280193236714975,
  'support': 161},
 '1': {'precision': 0.9134615384615384,
  'recall': 0.5337078651685393,
  'f1-score': 0.6737588652482269,
  'support': 178},
 '2': {'precision': 0.8528428093645485,
  'recall': 0.8793103448275862,
  'f1-score': 0.865874363327674,
  'support': 290},
 '3': {'precision': 0.0, 'recall': 0.0, 'f1-score': 0.0, 'support': 27},
 'accuracy': 0.7317073170731707,
 'macro avg': {'precision': 0.5700345849802372,
  'recall': 0.5551179065362984,
  'f1-score': 0.5419131380618496,
  'support': 656},
 'weighted avg': {'precision': 0.7509875629407707,
  'recall': 0.7317073170731707,
  'f1-score': 0.7197313330645136,
  'support': 656}}

In [79]:
len(preds), len(y)

(656, 656)